# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 8: Decision Trees

Implement a trainer for a single decision tree and an ensemble of decision tress, called Random Forest. Much of the code has already been completed in previous assignments. Once completed, you should have some learners to use for the Final Project.

The trainer implementation will happen in the following stages:

1. Create a ```NumericSplitter``` and a ```CategoricalSplitter``` which create a ```NumericSplit``` and a ```CategoricalSplit```, respecively. 
2. Create a ```DecisionTreeLearner``` which implements a recursive trainer given a list of possible splits
3. Add a ```predict``` method to the decision tree class that predicts for new examples
4. Using your ```DecisionTreeLearner``` as a weak learner, implement Bagging and a random subset of features to train a ```RandomForestTrainer```.
5. Implement the ```AUCMetric``` which helps you draw an ROC curve.
6. Apply $k$-fold Cross Validation to find the best parameters for two datasets. 

In [1]:
require './assignment_lib'

def load_german_credit_dataset; JSON.parse(File.read('german-credit.json')); end
def load_iris_dataset(); read_sparse_data_from_csv "iris"; end

puts "**Iris Dataset**"
puts "Features", load_iris_dataset()["features"], ""
puts "First example", load_iris_dataset()["data"].first

puts "\n\n**German Credit Dataset**"
puts "Features", load_german_credit_dataset()["features"], ""
puts "First example", load_german_credit_dataset()["data"].first

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

**Iris Dataset**
Features
["sepal_length", "sepal_width", "petal_length", "petal_width"]

First example
{"features"=>{"sepal_length"=>7.0, "sepal_width"=>3.2, "petal_length"=>4.7, "petal_width"=>1.4}, "label"=>1}


**German Credit Dataset**
Features
["checking_account", "loan_duration", "credit_history", "purpose", "credit_amount", "savings", "job_tenure", "installment_to_salary", "personal_status_gender", "other_debtors", "residence_tenure", "property", "age", "other_installments", "housing", "existing credits", "job", "dependents", "has_telephone", "is_foreign_worker"]

First example
{"id"=>0, "label"=>1, "features"=>{"checking_account"=>"(-inf,0)", "loan_duration"=>6.0, "credit_history"=>"critical account", "purpose"=>"radio/television", "credit_amount"=>1169.0, "savings"=>"none", "job_tenure"=>"[7,inf)", "installment_to_salary"=>4.0, "personal_status_gender"=>"m_single", "other_debtors"=>"none", "residence_tenure"=>4.0, "property"=>"real_estate", "age"=>67.0, "other_installments"=>

# Question 1

Copy the various pieces you have already implemented for decision trees

## Question 1.1 (1 Point)

Copy **your** implementation of ```class_distribution```, mean, and stdev from previous assignments

In [2]:
def class_distribution dataset
  # BEGIN YOUR CODE

    prob = Hash.new()
    if dataset.nil?
    else 
      i = 0
      distribution = []
      while i< dataset.length do
        distribution.append(dataset[i]["label"])
        i+=1
      end
      array = distribution.sort 
  
      type = array.select{ |x| array.count(x) >= 1}.uniq

      type.each do |n|
      
        prob.store(n,distribution.count(n)/(dataset.length).to_f)
      end
    end
    return prob
  
  #END YOUR CODE
end

def mean x
  # BEGIN YOUR CODE

  
   x.inject(0.0){|sum,i| sum + i }/x.length()  
  #END YOUR CODE
end

def stdev x
  # BEGIN YOUR CODE
  mu = mean x
  su = x.inject(0.0){|sum,i| sum + (i-mu)**2 }/(x.length()-1)
  #END YOUR CODE
end


:stdev

In [3]:
### TESTS ###
def test_d5732b
  iris = load_iris_dataset()
  t1_iris_dist = class_distribution iris["data"]
  t1_iris_num_classes = 3
  n = t1_iris_num_classes.times do |cls|
    assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
  end
  assert_equal t1_iris_num_classes, n
end
test_d5732b()

## Question 1.2 (1 Point)

Copy **your** implementation of ```entropy``` from previous assignments

In [4]:
def entropy dist
  # BEGIN YOUR CODE
  if dist.empty?
    en = 0
  else
    arr =(dist.to_a).transpose[1]
    if arr.include? 0 or arr.empty?
      en = 0
    else
      en = arr.inject(0.0){|sum,x| sum -x*Math.log(x) }
    end
  end
return en
  #END YOUR CODE
end

:entropy

In [5]:
### TESTS ###
def test_4b0b32()
  iris = load_iris_dataset()
  t2_iris_dist = class_distribution iris["data"]
  t2_iris_entropy = entropy t2_iris_dist
  assert_in_delta(1.0986122886681096, t2_iris_entropy, 1e-4)
  

  german_credit = load_german_credit_dataset()
  german_credit_dist = class_distribution german_credit["data"]
  german_credit_entropy = entropy german_credit_dist
  assert_in_delta(0.6108643020548935, german_credit_entropy, 1e-4)
end

test_4b0b32()

## Question 1.3 (1 Point)

Copy **your** implementation of ```information_gain``` from previous assignments

In [6]:
def information_gain h0, splits
  # BEGIN YOUR CODE
  i = 0
  name = splits.keys
  sum =0.0
  h = 0.0
  while i<splits.length do
    h += (splits[name[i]].length) * entropy( class_distribution(splits[name[i]]))
    sum += splits[name[i]].length
    i+=1
  end
  return h0- h/sum
  
  
  #END YOUR CODE
end

:information_gain

In [7]:
def test_21236c()
  iris = load_iris_dataset()
  t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
  t3_entropy = entropy(class_distribution(iris["data"]))

  assert_true(!t3_entropy.nil?)
end
test_21236c()

# Question 2 Splits

Implement a ```NumericSplit``` and a ```CategoricalSplit``` based on methods you implemented in previous assignments. Although much of the code will be reused, we will need to rearrange it.

The decision tree uses the ```Splitter``` to find and create the best ```Split``` for a feature. The ```Split``` holds the information needed to define the split and check whether an example shown go down one path or another. 


## Question 2.1 (5 Points)

Implement the ```CategoricalSplit```, which can split a dataset by the values of a categorical value. 

For example, consider splitting on the feature "f1". 

For the input as follows:
```ruby
[ 
    {"features" => {"f1" => "a"}}, 
    {"features" => {"f1" => "b"}},
    {"features" => {"f2" => "c"}}
]
```

The splits are stored in a hash where the key is a name of the path and the values are all examples under that path.
```ruby
{ 
    "f1 == 'a'" => [{"features" => {"f1" => "a"}}],
    "f1 == 'b'" => [{"features" => {"f1" => "b"}}]
}
```

Notes:

1. Skip any example which has a missing value for the features
1. Use the pattern ```@path_pattern % [@feature_name, feature_value]``` to assign a string-valued name to the split.

In [8]:
class CategoricalSplit
  attr_reader :feature_name
  
  def initialize fname
    @feature_name = fname
    @path_pattern = "%s == '%s'"
  end
  
  def to_s
    "Categorical[#{@feature_name}]"
  end

  def split_on_feature examples
    splits = Hash.new
 
    # BEGIN YOUR CODE
    ex = examples.select {|e|  e["features"][@feature_name].is_a? String }
    name = ex.map {|e|  e["features"][@feature_name] }.compact

   
    j = 0
    while j<name.length do
      i = 0    
      list = []
      list = examples.select{|e|  e["features"][@feature_name] == name[j]} 
      splits.store(@path_pattern % [@feature_name, name[j]],list)
      j+=1
    end
    
    
    #END YOUR CODE
    
    return splits
  end
end

:split_on_feature

The following test should output

```
Splits
["f1 == 'a'", "f1 == 'b'"]
[1, 1]
```

In [9]:
def test_af2e0c()  
  f1_split = CategoricalSplit.new "f1"

  example1 = {"features" => {"f1" => "a"}}
  example2 = {"features" => {"f1" => "b"}}
  example3_missing = {"features" => {"f2" => "c"}}
  examples = [example1, example2, example3_missing]
  
  splits = f1_split.split_on_feature examples
  puts "Splits", splits.keys  
  split_sizes = splits.values.collect {|v| v.size}
  puts split_sizes
  
  assert_equal 2, splits.size, "Returns all possible values for f1 feature"
  assert_equal 1, splits["f1 == 'a'"].size, "a"
  assert_equal 1, splits["f1 == 'b'"].size, "b"
end
test_af2e0c()

Splits
["f1 == 'a'", "f1 == 'b'"]
[1, 1]


The following test should output

```
Splits for german_credit.jobs
["job == 'skilled'", "job == 'unskilled'", "job == 'management'", "job == 'unemployed'"]
[630, 200, 148, 22]
```

In [10]:
def test_b08a8b()  
  job_split = CategoricalSplit.new "job"
  
  german_credit = load_german_credit_dataset()  
  examples = german_credit["data"]
  
  splits = job_split.split_on_feature examples
  puts "Splits for german_credit.jobs", splits.keys
  
  split_sizes = splits.values.collect {|v| v.size}
  puts split_sizes
  
  assert_equal 4, splits.size, "Returns all possible values for job feature"
  assert_equal 22, splits["job == 'unemployed'"].size, "unemployed"
  assert_equal 200, splits["job == 'unskilled'"].size, "unskilled"
end
test_b08a8b()

Splits for german_credit.jobs
["job == 'skilled'", "job == 'unskilled'", "job == 'management'", "job == 'unemployed'"]
[630, 200, 148, 22]


## Question 2.2 (4 Points)

Add a ```test``` method to your ```CategoricalSplit``` which considers an example and returns the string-valued name of the path to follow. If the example does not have a value for this feature, return ```nil```.

For example, for an example with feature "f1", defined as follows:

```ruby
{"features" => {"f1" => "a"}}
```

The ```test``` method returns the name of the appropriate path to follow based on the feature. Note that the string name is available through the ```@path_pattern``` member variable.

Output:
```ruby
"f1 == 'a'"
```

Notes:

1. Note that the categorical value could be generated dynamically. 

In [11]:
class CategoricalSplit
  def test example    
     if example["features"].keys.include? @feature_name
       vale = example["features"][@feature_name]
       path_name = @path_pattern % [@feature_name, vale]
     else
       path_name = nil
     end
    
  
    #END YOUR CODE
    
    return path_name
  end
end

:test

In [12]:
def test_746297()  
  f1_split = CategoricalSplit.new "f1"

  example1 = {"features" => {"f1" => "a"}}
  example2 = {"features" => {"f1" => "b"}}  
  example3 = {"features" => {"f2" => "c"}}
  
  assert_equal "f1 == 'a'", f1_split.test(example1), "Example 1"
  assert_equal "f1 == 'b'", f1_split.test(example2), "Example 2"  
  assert_equal nil, f1_split.test(example3), "Example 3"
end
test_746297()

## Question 2.3 (5 Points)

Implement ```NumericSplit``` which splits a dataset based on a provided numeric value. The ```split_on_feature``` method is very similar to the ```split_on_numeric_value``` from previous assignments. You may refactor **your** implementation of ```split_on_numeric_value```.

The splits are stored in a hash where the key is a name of the path and the values are all examples under that path. For example, consider the iris dataset, with a feature name "petal_length" and the value1 1.7. The ```split_on_feature``` method should return:

```ruby
{
    "petal_length >= 1.7" => [...],
    "petal_length < 1.7" => [...],
}
```

Notes:
1. For numeric values, any feature which is missing should be assumed to have a value of zero.
1. The path names are stored in a member array for numeric features because they are deterministic.

In [13]:
class NumericSplit
  attr_reader :feature_name, :split_point, :paths
  def initialize fname, value
    @feature_name = fname
    @split_point = value
    @split_point_str = "%.2g" % @split_point
    @paths = ["#{@feature_name} < #{@split_point_str}", "#{@feature_name} >= #{@split_point_str}"]
  end
  
  def to_s
    "Numeric[#{@feature_name} <=> #{@split_point_str}]"
  end

  def split_on_feature examples
    splits = Hash.new 

    # BEGIN YOUR CODE

    list = examples.select{|e| e["features"][@feature_name]>=@split_point }
    splits.store(@paths[1],list)
    splits.store(@paths[0],examples - list)
    #END YOUR CODE

    return splits
  end
end

:split_on_feature

The following test should output:

```
Splits
["petal_length >= 1.7", "petal_length < 1.7"]
[106, 44]
```

In [14]:
def test_c02859()  
  t21_split = NumericSplit.new "petal_length", 1.7
  
  iris = load_iris_dataset()
  t21_iris_splits = t21_split.split_on_feature iris["data"]
  puts "Splits", t21_iris_splits.keys  
  split_sizes = t21_iris_splits.values.collect {|v| v.size}
  puts split_sizes

  
  t21_split_sizes = t21_iris_splits.values.collect {|v| v.size}.sort
  t21_num_splits = 2
  assert_equal t21_num_splits, t21_split_sizes.size
  assert_equal 44, t21_split_sizes[0]
  assert_equal 106, t21_split_sizes[1]
end
test_c02859()

Splits
["petal_length >= 1.7", "petal_length < 1.7"]
[106, 44]


In [15]:
def test_17d874()
  iris = load_iris_dataset()
  split = NumericSplit.new "petal_length", 1.7
  t5_iris_splits = split.split_on_feature iris["data"]
  t5_split_sizes = t5_iris_splits.values.collect {|v| v.size}.sort

  # Checks the information gain for this split
  t5_iris_entropy = entropy(class_distribution(iris["data"]))
  t5_iris_information_gain = information_gain t5_iris_entropy, t5_iris_splits
  assert_in_delta 0.48280104455013506, t5_iris_information_gain, 5e-2
end
test_17d874()

## Question 2.3 (5 Points)

Add the ```test``` method to the ```NumericSplit``` which will return a string-valued path, corresponding to one of the values of the ```@paths``` array defined earlier. 

For example, consider the input as follows:

```ruby
{"features" => {"petal_length" => 0.7}}
```

The ```test``` method should return:

```ruby
"petal_length < 1.7" 
```

Notes:

1. If the feature value does not exist for the example, return ```nil```.

In [16]:
class NumericSplit
  def test example
    # BEGIN YOUR CODE
    
    if example["features"][@feature_name].nil?
      return nil
    elsif example["features"][@feature_name] < @split_point
      return "#{@feature_name} < #{@split_point_str}"
    elsif 
      example["features"][@feature_name] >= @split_point
      return "#{@feature_name} >= #{@split_point_str}"
    end
      
    
    #END YOUR CODE
  end
end

:test

In [17]:
def test_64e0d8()
  splitter = NumericSplit.new "petal_length", 1.7
  t22_x_left = {"features" => {"petal_length" => 0.7}}
  assert_equal splitter.paths[0], splitter.test(t22_x_left), "< 1.7"

  t22_x_right = {"features" => {"petal_length" => 1.71}}
  assert_equal splitter.paths[1], splitter.test(t22_x_right), ">= 1.7"

  t22_x_right_eq = {"features" => {"petal_length" => 1.7}}
  assert_equal splitter.paths[1], splitter.test(t22_x_right_eq), "= 1.7"
end
test_64e0d8()

In [18]:
def test_cf6969()
  splitter = NumericSplit.new "petal_length", 1.7
  x_missing = {"features" => {"petal_width" => 0.7}}
  assert_equal nil, splitter.test(x_missing), "Handle missing value"
end
test_cf6969()

# Question 3

Implement the ```CategoricalSplitter``` and ```NumericSplitter```. A ```Splitter``` checks that a ```Split``` can be applied, creates the ```Split```, and calculates the information gain. Note that the ```matches?``` function is provided for you and determines whether a split can apply. 

## Question 3.1 (5 points)

Implement the ```CategoricalSplit``` class, whose ```create_split``` method which takes the parent node's entropy, some examples and creates a ```CategoricalSplit``` on that feature name. It also calculates the information gain on the split.

For example, given a dataset and a feature name of "job", this should return the following:

```ruby
{"split" => CategoricalSplit(...), "information_gain" => 0.1234}
```

Notes:
1. You should call your ```information_gain``` function here.

In [19]:
class CategoricalSplitter
  def matches? examples, feature_name
    has_feature = examples.select {|r| r["features"].has_key? feature_name} 
    return false if has_feature.empty?    
    return has_feature.all? do |r| 
      r["features"].fetch(feature_name, 0.0).is_a?(String)
    end
  end
  
  def create_split examples, parent_entropy, feature_name
    # BEGIN YOUR CODE
  
    split = CategoricalSplit.new feature_name  
    splits = split.split_on_feature examples
    ig  = information_gain parent_entropy, splits
   
    #END YOUR CODE
    
    return {"split" => split, "information_gain" => ig}
  end
end

:create_split

In [20]:
def test_f251f0()
  german_credit = load_german_credit_dataset()
  examples = german_credit["data"]
  feature_name = "job"
  
  h0 = entropy(class_distribution(examples))
  splitter = CategoricalSplitter.new 
  split_result = splitter.create_split examples, h0, feature_name

  puts "Split Result:", split_result
  
  split = split_result["split"]
  info_gain = split_result["information_gain"]

  assert_not_nil split
  assert_in_delta 0.0009269851357646131, info_gain, 1e-2
end 
test_f251f0()

Split Result:
{"split"=>#<CategoricalSplit:0x000055d9032217f8 @feature_name="job", @path_pattern="%s == '%s'">, "information_gain"=>0.0009269851357646131}


## Question 3.2 (5 Points

Implement the ```NumericSplitter``` class. Given a feature, create_split finds the best feature value and creates the split. This behavior was implemented in a previous assignment, so adapt **your** ```find_split_point_numeric``` method to implement ```create_split```. 


```ruby
{"split" => NumericSplit(...), "information_gain" => 0.1234}
```

Notes:
1. In previous versions of the ```find_split_point_numeric```, we set missing values to zero. We will do the same thing here. However in the ```matches?``` function, we require at least one non-missing value. 
1. Return a ```nil``` if there is no available split point.

In [21]:
class NumericSplitter
  def matches? examples, feature_name
    has_feature = examples.select {|r| r["features"].has_key? feature_name} 
    return false if has_feature.empty?    
    return has_feature.all? do |r| 
      r["features"].fetch(feature_name, 0.0).is_a?(Numeric)
    end
  end
  
  def create_split examples, parent_entropy, feature_name    
    # BEGIN YOUR CODE
    if examples.empty?
      return nil
      
    else
      ex =  examples.select{|e| e["features"][feature_name].is_a? Numeric}
      points_list = ex.map{|e| e["features"][feature_name]}.sort
      if points_list.empty?
        return nil
        
      else
        ig_list = []
        i = 0
        while i < points_list.length do
          split = NumericSplit.new feature_name, points_list[i]
          split_list = split.split_on_feature examples
          ig_list.append( information_gain parent_entropy, split_list)
          i+=1
        end    
        ind = ig_list.each_with_index.max[1]
        if ind.nil?
          return nil
        else
            split = NumericSplit.new feature_name, points_list[ind]
    
            ig = ig_list[ind]

          #END YOUR CODE
            return {"split" => split, "information_gain" => ig}
        end
        
      end
  
    end
      
      
  end
end

:create_split

In [22]:
def test_f35008()
  iris = load_iris_dataset()
  t6_iris_entropy = entropy(class_distribution(iris["data"]))
  splitter = NumericSplitter.new 
  split_result = splitter.create_split iris["data"], t6_iris_entropy, "sepal_width"
  
  split = split_result["split"]
  info_gain = split_result["information_gain"]

  assert_in_delta 3.4, split.split_point, 1e-2, "Iris best split"

end
test_f35008()

In [23]:
def test_207f35()
  gc = load_german_credit_dataset()
  gc_entropy = entropy(class_distribution(gc["data"]))  
  splitter = NumericSplitter.new 
  split_result = splitter.create_split gc["data"], gc_entropy, "age"

  split = split_result["split"]
  info_gain = split_result["information_gain"]
  
  assert_not_nil split
  assert_in_delta 0.007817315714005901, info_gain, 1e-3
  assert_in_delta 26.0, split.split_point, 1e-2
end 
test_207f35()

In [24]:
def test_eedbae()
  parent_entropy = 1.0
  splitter = NumericSplitter.new 
  empty_examples = []
  split_result = splitter.create_split empty_examples, parent_entropy, "none"

  assert_nil split_result, "Empty examples"
  
  one_example = [{"features" => {"f1" => 1.0}, "label" => 1}]
  split_result = splitter.create_split one_example, parent_entropy, "one"

  assert_nil split_result, "One example"
end 
test_eedbae()

## Question 4

The core of the decision tree is the ```DecisionNode```, which is used in training and evaluation.

During training, the ```DecisionNode``` temporarily holds an array of examples, finds the best split and creates the splits. 

During evaluation, the node applies the test to find which of its children should be consulted for the leaf node. A leaf node is a node without any children.

## Question 4.1 (5 Points)

Implement the ```score``` method. When in a leaf node, the score for a particular class is calculated as the class distribution for the training data in that node. Because a decision tree is a multi-class classifier, we need to specify the class label when calling the ```score``` function. You should assume that there is no smoothing so if label isn't in the class distribution, the score is zero.

The score is the posterior class distribution given that an example is in the leaf $l$, defined as follows:

# $P(c \mid l) = \frac{ \left| \left\{x \mid c(x) = c \wedge x \in l \right\} \right| }{ \left| \left\{x\in l \right\} \right| }$

where $c(x)$ is the class label for example $x$.

Why is the score function for a leaf returning a constant and does not need to even look at the example?
  
**A**. Decision tree is overfitting by memorizing the training data and thus will not generalize

**B**. The code is wrong and it should be looking at the example

**C**. Decision tree assumes that all examples in the leaf have the same class distribution

**D**. It does look at the example, but it stores a class member variable so it does not need to be provided as an argument


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers is "Z", write the following:_

```ruby
def answer_ce174b()
    %w(Z)
end
```

In [25]:
def answer_673f8b()
  # BEGIN YOUR CODE

  %w(C)
  
  
  #END YOUR CODE
end

:answer_673f8b

In [26]:
assert_not_nil answer_673f8b()

In [27]:
class DecisionNode
  attr_reader  :children, :examples, :split, :node_entropy, :node_class_distribution
  
  def initialize examples
    @examples = examples
    @node_class_distribution = class_distribution examples    
    @node_entropy = entropy (@node_class_distribution)
    @children = Hash.new
  end
  
  def is_leaf?
    self.children.empty?
  end
      
  def score positive_class_label
    # BEGIN YOUR CODE
    
    if is_leaf?
      if @node_class_distribution.keys.include? positive_class_label
        return @node_class_distribution[positive_class_label]
      else 
        return 0.0
      end
    end
    
    
    #END YOUR CODE
  end

end

:score

In [28]:
def test_42e1b1()
  iris = load_iris_dataset()

  # Check the first split for iris
  examples = iris["data"]
  root = DecisionNode.new examples

  assert_true root.is_leaf?, "Single-node root is a leaf"
  
  assert_in_delta 0.3333, root.score(0), 1e-3, "Score for class 0"
  assert_in_delta 0.3333, root.score(1), 1e-3, "Score for class 1"
  assert_in_delta 0.3333, root.score(2), 1e-3, "Score for class 2"
  assert_in_delta 0.0, root.score(3), 1e-3, "Score for class 3 -- there is no class 3"  
end
test_42e1b1()

## Question 4.2 (10 Points)

Add ```all_possible_splits``` to the ```DecisionNode```. This considers an array of feature names and an array of splitters (```CategoricalSplitter``` or ```NumericSplitter```). Next, apply all splitters to the data within the node stored in ```@examples```, if they match, to each feature. Then, apply a set of filters:
1. Remove any split which is ```nil```
1. Remove any split where the information gain is not strictly $> 0$. 

Return all possible splits that meet the criteria. 

For example, given the iris dataset and the ```NumericSplitter```, return the following best splits for each feature:

```ruby
    [
        {"split"=>NumericSplit(feature_name: "petal_width", split_point: X), "information_gain" => 0.636514}, 
        {"split"=>NumericSplit(feature_name: "petal_length", split_point: X),  "information_gain"=>0.636514},
        {"split"=>NumericSplit(feature_name: "sepal_length", split_point: X), "information_gain"=>0.38624},
        ...
    ]
```

Note that the result above would be the same if we used passed both the ```CategoricalSplitter``` and ```NumericSplitter``` because only the ```NumericSplitter``` will ```matches?``` features in the iris dataset.

In [29]:
class DecisionNode
  def all_possible_splits feature_names, splitters
    all_splits = []
    
    # BEGIN YOUR CODE

 
   feature_names.each do |n|
        splitters.each do |w|
            split_result = w.create_split @examples, @node_entropy, n               
            if split_result!=nil and split_result["information_gain"] > 0 
                all_splits.append(split_result)
            end      
        end
    end

    
    
    #END YOUR CODE
    
    return all_splits
  end
end

:all_possible_splits

In [30]:
def test_e3c5cb()
  iris = load_iris_dataset()

  # Check the first split for iris
  feature_names = iris["features"]
  examples = iris["data"]
  root = DecisionNode.new examples
  
  splitters = [NumericSplitter.new]
  all_splits = root.all_possible_splits feature_names, splitters

  assert_equal 4, all_splits.size, "Calculate all possible splits"  
  sorted_splits = all_splits.sort_by {|split| split["information_gain"]}.reverse
  
  puts sorted_splits
  best_split = sorted_splits.first
  
  puts "Best split", best_split
  split = best_split["split"]
  info_gain = best_split["information_gain"]

  assert_not_nil split
  assert_in_delta 0.6365141682948128, info_gain, 1e-2, "Returns top info gain"
  assert_false %w(petal_length petal_width).index(split.feature_name).nil?, "Find one of the top features"
end
test_e3c5cb()

[{"split"=>#<NumericSplit:0x000055d903beb838 @feature_name="petal_width", @split_point=1.0, @split_point_str="1", @paths=["petal_width < 1", "petal_width >= 1"]>, "information_gain"=>0.6365141682948127}, {"split"=>#<NumericSplit:0x000055d903c600e8 @feature_name="petal_length", @split_point=3.0, @split_point_str="3", @paths=["petal_length < 3", "petal_length >= 3"]>, "information_gain"=>0.6365141682948127}, {"split"=>#<NumericSplit:0x000055d903d63c60 @feature_name="sepal_length", @split_point=5.6, @split_point_str="5.6", @paths=["sepal_length < 5.6", "sepal_length >= 5.6"]>, "information_gain"=>0.3862442664692114}, {"split"=>#<NumericSplit:0x000055d903c8a668 @feature_name="sepal_width", @split_point=3.4, @split_point_str="3.4", @paths=["sepal_width < 3.4", "sepal_width >= 3.4"]>, "information_gain"=>0.18570201019349375}]
Best split
{"split"=>#<NumericSplit:0x000055d903beb838 @feature_name="petal_width", @split_point=1.0, @split_point_str="1", @paths=["petal_width < 1", "petal_width >= 1

In [31]:
def test_377c12()
  german_credit = load_german_credit_dataset()

  # Check the first split for iris
  feature_names = german_credit["features"]
  examples = german_credit["data"]
  root = DecisionNode.new examples
  
  splitters = [CategoricalSplitter.new]
  all_splits = root.all_possible_splits feature_names, splitters

  assert_equal 13, all_splits.size, "Calculate all possible splits"  
  sorted_splits = all_splits.sort_by {|split| split["information_gain"]}.reverse
  
  puts sorted_splits
  best_split = sorted_splits.first
  
  puts "Best split", best_split
  split = best_split["split"]
  info_gain = best_split["information_gain"]

  assert_not_nil split
  assert_in_delta 0.06566796091172744, info_gain, 1e-2, "Returns top info gain"
  assert_false %w(checking_account).index(split.feature_name).nil?, "Find one of the top features"
end
test_377c12()

[{"split"=>#<CategoricalSplit:0x000055d903a4bb90 @feature_name="checking_account", @path_pattern="%s == '%s'">, "information_gain"=>0.06566796091172744}, {"split"=>#<CategoricalSplit:0x000055d903e0c4a0 @feature_name="credit_history", @path_pattern="%s == '%s'">, "information_gain"=>0.03023355461426125}, {"split"=>#<CategoricalSplit:0x000055d903df3888 @feature_name="savings", @path_pattern="%s == '%s'">, "information_gain"=>0.01948760776931846}, {"split"=>#<CategoricalSplit:0x000055d903aab5e0 @feature_name="purpose", @path_pattern="%s == '%s'">, "information_gain"=>0.017254887066567637}, {"split"=>#<CategoricalSplit:0x000055d903c62028 @feature_name="property", @path_pattern="%s == '%s'">, "information_gain"=>0.011773233742666478}, {"split"=>#<CategoricalSplit:0x000055d903a65cc0 @feature_name="job_tenure", @path_pattern="%s == '%s'">, "information_gain"=>0.009081837924813763}, {"split"=>#<CategoricalSplit:0x000055d903bb9658 @feature_name="housing", @path_pattern="%s == '%s'">, "informati

## Question 4.3 (5 Points)

Given a specific split, possibly created from ```all_possible_splits``` above, implement ```split_node!```. Ruby's convention of adding a "!" to a method indicates that the method changes the state of the object. In this case, the ```split_node!``` method calls the ```split_on_feature``` method of the split and creates children ```DecisionNode```s. The ```@children``` member variable is a hash which maps each path calculated by the split to ```DecisionNode```. 

Given input examples as follows:
```ruby
[
    {"features" => {"f1" => "a", "f2" => "d"}},
    {"features" => {"f1" => "a", "f2" => "c"}},
    {"features" => {"f1" => "b", "f2" => "c"}}
]
```

Children should be as follows:
```ruby
{
    "f1 == 'a'" => DecisionNode(...),
    "f1 == 'b'" => DecisionNode(...)
}
```

Note:
1. The ```@examples``` data held by the node is freed after splitting to reduce memory consumption.

In [32]:
class DecisionNode
  def split_node! split    
    @split = split
    # BEGIN YOUR CODE


    b =  @split.split_on_feature @examples

    b.keys.each do |n|
       @children.store(n,DecisionNode.new( b[n]))  
    end

    return @children
    #END YOUR CODE
    
    @examples = nil
  end
end

:split_node!

The following test should output:
```
{"f1 == 'a'"=>#<DecisionNode:0x000056518b298708 @examples=[{"features"=>{"f1"=>"a", "f2"=>"d"}}, {"features"=>{"f1"=>"a", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=-0.0, @children={}>, "f1 == 'b'"=>#<DecisionNode:0x000056518b298488 @examples=[{"features"=>{"f1"=>"b", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=-0.0, @children={}>}
```

In [33]:
def test_cfc360()  
  # Check the first split for iris
  example1 = {"features" => {"f1" => "a", "f2" => "d"}}
  example2 = {"features" => {"f1" => "a", "f2" => "c"}}
  example3 = {"features" => {"f1" => "b", "f2" => "c"}}
  
  examples = [example1, example2, example3]
  
  root = DecisionNode.new examples  
  split = CategoricalSplit.new "f1"  
  root.split_node! split

  children = root.children
  puts children
  assert_equal 2, children.size, "Creates 2 children"  
  assert_not_same examples, children["f1 == 'a'"].examples, "Child a did not see a subset of data"
  assert_not_same examples, children["f1 == 'b'"].examples, "Child b did not see a subset of data"
  assert_equal 2, children["f1 == 'a'"].examples.size, "Child a"
  assert_equal 1, children["f1 == 'b'"].examples.size, "Child b"
end
test_cfc360()

{"f1 == 'a'"=>#<DecisionNode:0x000055d90327bf28 @examples=[{"features"=>{"f1"=>"a", "f2"=>"d"}}, {"features"=>{"f1"=>"a", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=0.0, @children={}>, "f1 == 'b'"=>#<DecisionNode:0x000055d90327b8e8 @examples=[{"features"=>{"f1"=>"b", "f2"=>"c"}}], @node_class_distribution={nil=>1.0}, @node_entropy=0.0, @children={}>}


# Question 5

With the ```DecisionNode``` complete, we are ready to build the decision tree trainer. The decision tree is an example of a ```Learner``` which we are defining below. The ```Learner``` interface will enable us to compare several different models and combinations of models. For this and all future assignments, we will implement new algorithms as a ```Learner```. The interface is defined below. Note that Ruby does not have an explicit interface, so we use a mixin module.

In [34]:
module Learner  
  attr_reader :parameters
  def train train_dataset    
  end
  def predict example
  end
  def evaluate eval_dataset
  end
end
  

:evaluate

The contract for a ```Learner``` is defined as follows:

* ```parameters```: A hash containing parameter values used for training. Keep only model hyper-parameters in this hash. 
* ```train```: Trains the model and keeps any model state in the object. Post training, the learner can be used as a trained model. The same learner can be re-trained if needed.
* ```predict```: Predicts on a single example, expects the model to be trained first. Assumes the model is a binary classifier and returns a single score related to the positive class. 
* ```evaluate```: Evaluates the model on a full dataset and returns an array of (score, label) pairs.


In ```evaluate```, the output is as follows:

```ruby
[
    [0.12345, 0],
    [0.4562, 1]
]
````

where the first entry in each element the score (output of predict) and the second is the label. Use 1 for positive and 0 for negative regardless of how the class is defined in the dataset.

## Question 5.1 (10 points)

The ```train``` method for a decison tree is simple.  It calls a recursive method ```grow_tree``` which starts at a ```parent``` and proceeds to grow the parent by considering all possible splits, take the best, and create children. Then, it tries to grow each child on that child's subset of the data. 

As a recursive function, ```grow_tree``` has 3 termination criteria:

1. The tree is too deep--no more remaining depth
2. No more splits available
3. No more than ```min_size``` examples in the parent.

A summary method method has been provided for you to print the resulting tree. For a ```max_depth``` of 1, this should produce a single leaf node. On the iris dataset, this is as follows:

```json
{
  "leaf": true,
  "class_distribution": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  }
}
```

For a ```max_depth``` of 2, this should produce a tree with exactly one split. On the iris dataset, this is as follows:

```json
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": true,
      "class_distribution": {
        "1": 0.5,
        "2": 0.5
      }
    }
  }
}
```

Notes:
1. We are implementing a fixed height (depth) tree. This is necessary both to prevent overflow but we will also use it for testing. Keep in mind that depth is **not** a typical hyperparameter. We are just using it to simplify the calculation. In practice, set the depth to a really large number and use the ```min_size``` as the true hyperparameter.

In [35]:
class DecisionTreeLearner
  include DecisionTreeHelper
  include Learner  
# ################################################### This is a debug
#   attr_reader  :examples
# #####################################################   
  attr_reader :root
# ################################################### This is a debug
#   attr_reader :splitters, :parameters, :positive_class_label, :feature_names, :children, :examples
# #####################################################   
  def initialize positive_class_label, min_size: 10, max_depth: 50
    @splitters = [CategoricalSplitter.new, NumericSplitter.new]
    @parameters = {"min_size" => min_size, "max_depth" => max_depth}
    @positive_class_label = positive_class_label

    
  end
    
  def train dataset
    @feature_names = dataset["features"]
    examples = dataset["data"]        
    @root = DecisionNode.new examples
    grow_tree @root, @parameters["max_depth"]  

  end

  def grow_tree parent, remaining_depth
    # BEGIN YOUR CODE
    
    if  parent.examples.size >= @parameters["min_size"] and remaining_depth > 1 
      

          all_splits = parent.all_possible_splits @feature_names, @splitters
          sorted_splits = all_splits.sort_by! {|split| split["information_gain"]}.reverse

          if !all_splits.empty?  and !all_splits.nil?
                
                  
                  a = sorted_splits.map{|split| split["information_gain"]}
           
                  index = a.each_with_index.max[1]
      
       
                  best_split = sorted_splits[index]
                  split = best_split["split"]

                  c = parent.split_node! split
           
                  if parent.is_leaf?
                        return parent
                  else
                        remaining_depth += -1
                        parent.children.each do |k,v|
                          self.grow_tree(v,remaining_depth)
                        end
                  end
          else
            return parent

          end
    else 
      
      return @parent
    end

    #END YOUR CODE
  end
end

:grow_tree

The following test should output:

```
Root Only Tree
{
  "leaf": true,
  "class_distribution": {
    "1": 0.3333333333333333,
    "2": 0.3333333333333333,
    "0": 0.3333333333333333
  }
}
```

In [36]:
def test_594f6e()
  iris = load_iris_dataset()
  
  model = DecisionTreeLearner.new 1, min_size: 10, max_depth: 1
  model.train iris
  tree = model.root

  puts "Root Only Tree", model.to_s

  assert_true tree.children.empty?
  assert_true tree.is_leaf?
end

test_594f6e()

Root Only Tree
{
  "leaf": true,
  "class_distribution": {
    "0": 0.3333333333333333,
    "1": 0.3333333333333333,
    "2": 0.3333333333333333
  }
}


The following test should output:
```
Two-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": true,
      "class_distribution": {
        "1": 0.5,
        "2": 0.5
      }
    }
  }
}
```

In [37]:
def test_1c728f()
  iris = load_iris_dataset()
  model = DecisionTreeLearner.new 1, min_size: 10, max_depth: 2
  model.train iris
  root = model.root

  puts "Two-level Tree", model.to_s

  assert_false root.is_leaf?
  assert_not_nil root.split
  assert_equal 2, root.children.size
  assert_true(root.children.values.all? {|leaf| leaf.is_leaf?})
end
test_1c728f()

Two-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": true,
      "class_distribution": {
        "1": 0.5,
        "2": 0.5
      }
    }
  }
}


The following test should output:

```
Multi-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": false,
      "split": "Numeric[petal_width <=> 1.8]",
      "children": {
        "petal_width < 1.8": {
          "leaf": false,
          "split": "Numeric[petal_length <=> 5]",
          "children": {
            "petal_length < 5": {
              "leaf": false,
              "split": "Numeric[petal_width <=> 1.7]",
              "children": {
                "petal_width < 1.7": {
                  "leaf": true,
                  "class_distribution": {
                    "1": 1.0
                  }
                },
                "petal_width >= 1.7": {
                  "leaf": true,
                  "class_distribution": {
                    "2": 1.0
                  }
                }
              }
            },
            "petal_length >= 5": {
              "leaf": true,
              "class_distribution": {
                "1": 0.3333333333333333,
                "2": 0.6666666666666666
              }
            }
          }
        },
        "petal_width >= 1.8": {
          "leaf": false,
          "split": "Numeric[petal_length <=> 4.9]",
          "children": {
            "petal_length < 4.9": {
              "leaf": true,
              "class_distribution": {
                "2": 0.6666666666666666,
                "1": 0.3333333333333333
              }
            },
            "petal_length >= 4.9": {
              "leaf": true,
              "class_distribution": {
                "2": 1.0
              }
            }
          }
        }
      }
    }
  }
}
```

In [38]:
def test_1e9f1b()
  iris = load_iris_dataset()
  model = DecisionTreeLearner.new 1, min_size: 25, max_depth: 10
  model.train iris
  root = model.root

  puts "Multi-level Tree", model.to_s

  assert_false root.is_leaf?
  assert_not_nil root.split
  assert_equal 2, root.children.size
end
test_1e9f1b()

Multi-level Tree
{
  "leaf": false,
  "split": "Numeric[petal_length <=> 3]",
  "children": {
    "petal_length < 3": {
      "leaf": true,
      "class_distribution": {
        "0": 1.0
      }
    },
    "petal_length >= 3": {
      "leaf": false,
      "split": "Numeric[petal_width <=> 1.8]",
      "children": {
        "petal_width < 1.8": {
          "leaf": false,
          "split": "Numeric[petal_length <=> 5]",
          "children": {
            "petal_length < 5": {
              "leaf": false,
              "split": "Numeric[petal_width <=> 1.7]",
              "children": {
                "petal_width < 1.7": {
                  "leaf": true,
                  "class_distribution": {
                    "1": 1.0
                  }
                },
                "petal_width >= 1.7": {
                  "leaf": true,
                  "class_distribution": {
                    "2": 1.0
                  }
                }
              }
            },
            "p

## Question 5.2 (5 Points)

Implement ```find_leaf``` which maps an example to an appropriate leaf node. By recursively applying the split tests, find the leaf node. That leaf will be used to calculate the score. 

The ```find_leaf``` method is recursive given a node. It has 2 termination conditions:

1. Node is a leaf
2. Node would have applied a split but there is no child. This happens if a new categorical value was added.


In [39]:
class DecisionTreeLearner
  attr_accessor :positive_class_label
  def predict example
    leaf = find_leaf @root, example
 
    return leaf.score @positive_class_label
  end

  def evaluate eval_dataset
    examples = eval_dataset["data"]
    
    examples.map do |example|

      score = predict(example)
      
      label = example["label"] == @positive_class_label ? 1 : 0
      [score, label]
    end
  end

  def find_leaf node, example
    # BEGIN YOUR CODE
      root = node
      pattern = "%s == '%s'"



     while !root.children.nil? and !root.is_leaf?          
          n = root.split.feature_name  
          data = example["features"][n]

          number = root.children.keys[0].split[2]

#           if eval(number).is_a? Numeric          ###### this is for numerica category
          if data.is_a? Numeric
              if data < number.to_f
                next_key = "#{n} < " + number
              else
                next_key = "#{n} >= " + number
              end


          else
  
#               root.children.keys.each do |o|
#                   string = o.split[2]
#                   puts data, string,"test"
#                   if eval(string).is_a? String and data.is_a? String
#                     puts "this is a debug"
#                   end
#                   puts "'here'"
#                   if eval(string) == data
#                     next_key = o
#                     puts next_key
#                   end
#                   break if eval(string) == data
           
#               end 
              next_key = pattern%[n,data]


             
          end
          if root.children.nil? or root.is_leaf? 
              root = root
          else
              root = root.children[next_key]

          end
        
      end
  
      return root

    #END YOUR CODE
  end
end

:find_leaf

In [40]:
def test_e35c4e()
  iris = load_iris_dataset()
  
  ##Force decision tree to only have 'petal_length' feature
  iris["features"] = %w(petal_length)
  
  learner = DecisionTreeLearner.new 1, min_size: 10, max_depth: 2
  learner.train iris  
  
  x1 = {"features" => {"petal_length" => 1.2}}
  assert_in_delta 0.0, learner.predict(x1), 1e-3, "example 1, class 1"

  x2 = {"features" => {"petal_length" => 3.1}}
  assert_in_delta 0.5, learner.predict(x2), 1e-3, "example 2, class 1"
  
  learner.positive_class_label = 0
  x3 = {"features" => {"petal_length" => 1.2}}
  assert_in_delta 1.0, learner.predict(x3), 1e-3, "example 3, class 0"
end

test_e35c4e()

## Question 5.3 (5 Points)

Reuse your ```AUCMetric``` here calculates the AUC and plots the curve for a classifier.

The return format for the ```roc_curve``` function is defined as follows:

```ruby
#False positive rates
fp_rates = [0.0, 0.01, 0.02, ..., 1.0]

#True positive rates
tp_rates = [0.0, 0.01, 0.02, ..., 1.0]

#AUC Value
auc = 0.5

return [fp, tp, auc]
```

When plotted, you should see a curve like this:

![example_roc_curve](./roc_curve.png)




In [41]:
module Metric
  def apply scores
  end
end

:apply

In [42]:
def num_negatives scores
  # BEGIN YOUR CODE

  return scores.length - num_positives(scores)
  
  #END YOUR CODE
end

def num_positives scores
  # BEGIN YOUR CODE

  i = 0
  sum = 0
  while i< scores.length do
    if scores[i][1]>0
      sum += 1
    end
    i+=1
  end
  return sum
  
  #END YOUR CODE
end

:num_positives

In [43]:
class AUCMetric 
  include Metric
  def roc_curve(scores)
    # BEGIN YOUR CODE
    fp_rates = [0.0]
    tp_rates = [0.0]
    auc = 0.0

    # BEGIN YOUR CODE


    t_list = scores.sort_by{|e| [-e[0]]}

    i = 0
    total_pos = num_positives scores
    total_neg = num_negatives scores
    p = 0.0
    n = 0.0

    t_list.each do |e|

      if e[1] == 1
        p +=1.0   
        fp_rates.append(fp_rates[-1])      
        tp_rates.append(p/total_pos)
      else
        n +=1.0 
        fp_rates.append(n/total_neg)      
        tp_rates.append(tp_rates[-1])
      end

    end

    i = 1
    while i < (tp_rates.length) do
        auc+= 0.5*(fp_rates[i] -fp_rates[i-1])*(tp_rates[i] + tp_rates[i-1]  )
      i+=1
    end
  
    
    #END YOUR CODE
    return [fp_rates, tp_rates, auc]
  end
  
  def apply scores
    fp, tp, auc = roc_curve scores
    return auc
  end
end

:apply

In [44]:
##### the follwing cell takes about 8 minutes.

In [45]:
def test_99112c()
  german_credit = load_german_credit_dataset()
  examples = german_credit["data"]
  learner = DecisionTreeLearner.new 1, min_size: 100, max_depth: 10
  learner.train german_credit

  scores = learner.evaluate german_credit
  metric = AUCMetric.new
  fp, tp, auc = metric.roc_curve scores
  
  puts "AUC = #{auc}"
  
  plot = Daru::DataFrame.new({x: fp, y: tp}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "False Positive Rate"
    plot.y_label "True Positive Rate"
  end
  plot.add(:line, [0,1], [0,1]).color(:gray)
  plot.show()
end

test_99112c()

AUC = 0.7965619047619045


#<Nyaplot::Frame:0x000055d903f00c80 @properties={:panes=>[#<Nyaplot::Plot:0x000055d903c5d2d0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000055d903f083b8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"320b9940-6b62-41f6-8409-4b59ea63e491"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>, #<Nyaplot::Diagram:0x000055d903f01cc0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>:gray}, :data=>"1d748c17-7e7d-40a0-89cd-e27f1cac7675"}, @xrange=[0, 1], @yrange=[0, 1]>], :options=>{:x_label=>"False Positive Rate", :y_label=>"True Positive Rate", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>], :data=>{"320b9940-6b62-41f6-8409-4b59ea63e491"=>#<Nyaplot::DataFrame:0x000055d902560bb8 @name="320b9940-6b62-41f6-8409-4b59ea63e491", @rows=[{:x=>0.0, :y=>0.0}, {:x=>0.0, :y=>0.0014285714285714286}, {:x=>0.0, :y=>0.002857142857142857}, {:x=>0.0, :y=>0.004285714285714286}, {:x=>0.0, :y=>0.005714285714285714}, {:x=>0.0, :y=>0.007142857142857143}, {:x=>0.0, :y=>0.008571428571428572}, {:x=>0.0, :y=>0.01}, {:x=>0.0, :y=>0.011428571428571429}, {:x=>0.0, :y=>0.012857142857142857}, {:x=>0.0, :y=>0.014285714285714285}, {:x=>0.0, :y=>0.015714285714285715}, {:x=>0.0, :y=>0.017142857142857144}, {:x=>0.0, :y=>0.018571428571428572}, {:x=>0.0, :y=>0.02}, {:x=>0.0, :y=>0.02142857142857143}, {:x=>0.0, :y=>0.022857142857142857}, {:x=>0.0, :y=>0.024285714285714285}, {:x=>0.0, :y=>0.025714285714285714}, {:x=>0.0, :y=>0.027142857142857142}, {:x=>0.0, :y=>0.02857142857142857}, {:x=>0.0, :y=>0.03}, {:x=>0.0, :y=>0.03142857142857143}, {:x=>0.0, :y=>0.032857142857142856}, {:x=>0.0, :y=>0.03428571428571429}, {:x=>0.0, :y=>0.03571428571428571}, {:x=>0.0, :y=>0.037142857142857144}, {:x=>0.0, :y=>0.03857142857142857}, {:x=>0.0, :y=>0.04}, {:x=>0.0, :y=>0.041428571428571426}, {:x=>0.0, :y=>0.04285714285714286}, {:x=>0.0, :y=>0.04428571428571428}, {:x=>0.0, :y=>0.045714285714285714}, {:x=>0.0, :y=>0.047142857142857146}, {:x=>0.0, :y=>0.04857142857142857}, {:x=>0.0, :y=>0.05}, {:x=>0.0, :y=>0.05142857142857143}, {:x=>0.0, :y=>0.05285714285714286}, {:x=>0.0, :y=>0.054285714285714284}, {:x=>0.0, :y=>0.055714285714285716}, {:x=>0.0, :y=>0.05714285714285714}, {:x=>0.0, :y=>0.05857142857142857}, {:x=>0.0, :y=>0.06}, {:x=>0.0, :y=>0.06142857142857143}, {:x=>0.0, :y=>0.06285714285714286}, {:x=>0.0, :y=>0.06428571428571428}, {:x=>0.0, :y=>0.06571428571428571}, {:x=>0.0, :y=>0.06714285714285714}, {:x=>0.0, :y=>0.06857142857142857}, {:x=>0.0, :y=>0.07}, {:x=>0.0, :y=>0.07142857142857142}, {:x=>0.0, :y=>0.07285714285714286}, {:x=>0.0, :y=>0.07428571428571429}, {:x=>0.0, :y=>0.07571428571428572}, {:x=>0.0, :y=>0.07714285714285714}, {:x=>0.0, :y=>0.07857142857142857}, {:x=>0.0, :y=>0.08}, {:x=>0.0, :y=>0.08142857142857143}, {:x=>0.0, :y=>0.08285714285714285}, {:x=>0.0, :y=>0.08428571428571428}, {:x=>0.0, :y=>0.08571428571428572}, {:x=>0.0, :y=>0.08714285714285715}, {:x=>0.0, :y=>0.08857142857142856}, {:x=>0.0, :y=>0.09}, {:x=>0.0, :y=>0.09142857142857143}, {:x=>0.0, :y=>0.09285714285714286}, {:x=>0.0, :y=>0.09428571428571429}, {:x=>0.0, :y=>0.09571428571428571}, {:x=>0.0, :y=>0.09714285714285714}, {:x=>0.0, :y=>0.09857142857142857}, {:x=>0.0, :y=>0.1}, {:x=>0.0, :y=>0.10142857142857142}, {:x=>0.0, :y=>0.10285714285714286}, {:x=>0.0, :y=>0.10428571428571429}, {:x=>0.0, :y=>0.10571428571428572}, {:x=>0.0, :y=>0.10714285714285714}, {:x=>0.0, :y=>0.10857142857142857}, {:x=>0.0, :y=>0.11}, {:x=>0.0, :y=>0.11142857142857143}, {:x=>0.0, :y=>0.11285714285714285}, {:x=>0.0, :y=>0.11428571428571428}, {:x=>0.0, :y=>0.11571428571428571}, {:x=>0.0, :y=>0.11714285714285715}, {:x=>0.0, :y=>0.11857142857142858}, {:x=>0.0, :y=>0.12}, {:x=>0.0, :y=>0.12142857142857143}, {:x=>0.0, :y=>0.12285714285714286}, {:x=>0.0, :y=>0.12428571428571429}, {:x=>0.0, :y=>0.12571428571428572}, {:x=>0.0, :y=>0.12714285714285714}, {:x=>0.0, :y=>0.12857142857142856}, {:x=>0.0, :y=>0.13}, {:x=>0.0, :y=>0.13142857142857142}, {:x=>0.0, :y=>0.13285714285714287}, {:x=>0.0, :y=>0

#<CZTop::Socket::PUB:0x55d902c76d70 last_endpoint="tcp://127.0.0.1:41235">

# Question 6

Using the height-limited Decision Tree Learner as a weak learner (i.e., a Decision Stump), we will now implement the ```RandomForestLearner```. The random forest classifier requires a seed, which is provided below.

## Question 6.1 (10 points)
 
The Random Forest algorithm combines Bagging with random features. Given a dataset, we will first identify a random subset of 3 features from the dataset. 


Next, implement Bagging which is to create bootstrap sample. A bootstrap sample is a sample of the same size as the original dataset, but each example is sampled with uniform probability with replacement. Sampling with replacement will result in some of the examples appearing more than once in the final dataset. 

The ```random_forest_dataset``` should return a new dataset where ```data``` and ```features``` have been changed based on the bootstrapping and random features logic. Make sure to make a _deep copy_ rather than a shallow copy of the dataset. Hint: use the ```clone``` method appropriately.

For example, given a dataset with 4 features, we expect a dataset with 3 features, defined as follows:

```ruby
{
    "labels" => [...],
    "features" => ["x2", "x3", "x4"],
    "data" => [
        {"id"=>92, "features"=>{"x2"=>92, "x4"=>92, "x3"=>92}}, 
        {"id"=>26, "features"=>{"x2"=>26, "x4"=>26, "x3"=>26}}, 
        {"id"=>33, "features"=>{"x2"=>33, "x4"=>33, "x3"=>33}}, 
        ...
    ]
}
```


Notes:

1. Because we are using random numbers it is important that you call ```rng.rand(...)``` the right number of times. Using some other random number generator or even just one extra call will cause the tests not to pass. See Ruby's [Random](https://ruby-doc.org/core-2.5.4/Random.html) class for more details.

In [46]:
SEED = 'eifjcchdivlbcbflbgblfgukbtkhvejvtkevfbtetjnl'.to_i(26)

3073016676488

In [47]:
def random_features_subset dataset, rng
  num_features = 3
  feature_list = dataset["features"].sample(num_features, random: rng)  
end

def random_forest_dataset dataset, rng
  feature_list = random_features_subset dataset, rng
  examples = dataset["data"]
  new_dataset = nil
  
  # BEGIN YOUR CODE

  
  dat = examples.clone
  new_dataset = Hash.new
  origin_feature = dataset["features"].clone
  a = dat.clone
  remove_list = origin_feature - feature_list
  
  removed_dataset = []
  a.each do |n|
    h = Hash.new
    nn = n.clone


    c = nn["features"].clone
    remove_list.each{|e| c.delete(e)}
    h["features"] = c
 
    nn.delete("features")

    new = nn.merge h

    removed_dataset.append(new)

  end
  

  
  new_dataset["features"] = feature_list
  new_dataset["data"] = []
  removed_dataset.size.times do 
      new_dataset["data"].append(removed_dataset.sample(1, random: rng)[0])
  end
  #END YOUR CODE

  return new_dataset
end

:random_forest_dataset

In [48]:
def test_8b4e54()
  rng = Random.new(SEED)
  
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => id * 1.0, "x2" => id, "x3" => id, "x4" => id}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  assert_not_same dataset, rf_dataset, "Must create a new dataset"
  assert_not_same dataset["features"], rf_dataset["features"], "Must create new features"
  assert_not_same dataset["data"], rf_dataset["data"], "Must create new examples"  
  assert_not_same dataset["data"][92], rf_dataset["data"][0], "Must create new example 0 rather than reusing #92"   
  assert_not_same dataset["data"][92]["features"], rf_dataset["data"][0]["features"], "Must create new example features"     
end
test_8b4e54()

In [49]:
def test_61dee6()
  rng = Random.new(SEED)
  
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => id * 1.0, "x2" => id, "x3" => id, "x4" => id}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  features = rf_dataset["features"].sort
  puts "Sampled features", features
  
  assert_equal 3, features.size
  assert_equal "x2", features[0]  
  assert_equal "x4", features[2]  
end
test_61dee6()

Sampled features
["x2", "x3", "x4"]


The following test should output:

```
Sampled features
["x2", "x3", "x4"]
[{"id"=>92, "features"=>{"x2"=>0.7382309556062473, "x4"=>0.8321339621196061, "x3"=>0.4902051538880293}}, {"id"=>26, "features"=>{"x2"=>0.9058511884689645, "x4"=>0.14862808413212236, "x3"=>0.6480159304053194}}, {"id"=>33, "features"=>{"x2"=>0.8779817106571303, "x4"=>0.052513163798546425, "x3"=>0.5935117988751606}}, {"id"=>23, "features"=>{"x2"=>0.8127461649463698, "x4"=>0.11281946181217362, "x3"=>0.9818758966334915}}, {"id"=>55, "features"=>{"x2"=>0.6965391901897178, "x4"=>0.059710713558340456, "x3"=>0.37992498973029476}}]
Expect ID 91 is duplicated
```

In [50]:
def test_e26103()
  rng = Random.new(SEED)
  rng2 = Random.new(SEED)  
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => rng2.rand, "x2" => rng2.rand, "x3" => rng2.rand, "x4" => rng2.rand}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  
  features = rf_dataset["features"].sort
  puts "Sampled features", features
  
  rf_examples = rf_dataset["data"]
  puts rf_examples[0,5]
  assert_equal 100, rf_examples.size
  assert_equal 92, rf_examples[0]["id"]  
  assert_in_delta 0.7382309556062473, rf_examples[0]["features"]["x2"], 1e-3, "Must copy original example values"
  assert_equal 23, rf_examples[3]["id"]
  assert_in_delta 0.11281946181217362, rf_examples[3]["features"]["x4"], 1e-3, "Must copy original example values"
  
  duplicated_ids = rf_examples.group_by {|e| e["id"]}.select {|k,v| v.size > 1}.to_h.keys.sort
  
  puts "Expect ID #{duplicated_ids.last} is duplicated"
  assert_equal 25, duplicated_ids.size
  assert_equal 91, duplicated_ids.last
end
test_e26103()

Sampled features
["x2", "x3", "x4"]
[{"id"=>92, "features"=>{"x2"=>0.7382309556062473, "x3"=>0.4902051538880293, "x4"=>0.8321339621196061}}, {"id"=>26, "features"=>{"x2"=>0.9058511884689645, "x3"=>0.6480159304053194, "x4"=>0.14862808413212236}}, {"id"=>33, "features"=>{"x2"=>0.8779817106571303, "x3"=>0.5935117988751606, "x4"=>0.052513163798546425}}, {"id"=>23, "features"=>{"x2"=>0.8127461649463698, "x3"=>0.9818758966334915, "x4"=>0.11281946181217362}}, {"id"=>55, "features"=>{"x2"=>0.6965391901897178, "x3"=>0.37992498973029476, "x4"=>0.059710713558340456}}]
Expect ID 91 is duplicated


In [51]:
def test_5682b7()
  rng = Random.new(SEED)
  examples = Array.new 100 do |id|
    {"id" => id, "features" => {"x1" => id * 1.0, "x2" => id, "x3" => id, "x4" => id}}
  end
  dataset = {"features" => %w(x1 x2 x3 x4), "data" => examples}
  
  rf_dataset = random_forest_dataset dataset, rng
  
  features = rf_dataset["features"].sort
  rf_examples = rf_dataset["data"]
  
  assert_equal 100, rf_examples.size
  checked = 0
  rf_examples.each do |e| 
    assert_equal features, e["features"].keys.sort, "Expected only subset of features"
    checked += 1
  end
  assert_equal 100, checked
end
test_5682b7()

## Question 6.2 (2 Points)

Implement the ```train``` method in the ```RandomForestLearner```, which simply calls train on each of the ```@trees``` with the ```random_forest_dataset```.

In [52]:
class RandomForestLearner
  include Learner  
  attr_reader :trees
  
  def initialize positive_class_label, num_trees: 10, min_size: 10, max_depth: 50
    @parameters = {"num_trees" => num_trees, "min_size" => min_size, "max_depth" => max_depth}
    @positive_class_label = positive_class_label
    tree_parameters = @parameters.clone.delete :num_trees
    
    @trees = Array.new(num_trees) do |i| 
      DecisionTreeLearner.new @positive_class_label, min_size: min_size, max_depth: max_depth
    end
  end
  
  def to_s
    JSON.pretty_generate(@trees.collect {|t| t.summarize_node t.root})
  end
  
  def train dataset
    rng = Random.new SEED
    
    # BEGIN YOUR CODE
    rf_dataset = random_forest_dataset dataset, rng

    @trees.each do |n|
     n.train rf_dataset
   end
    
    #END YOUR CODE
  end
end
  

:train

The following test should output:
```
Random Forest 3 trees, 2 levels
[
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_width >= 1": {
        "leaf": true,
        "class_distribution": {
          "2": 0.5555555555555556,
          "1": 0.4444444444444444
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_width >= 1": {
        "leaf": true,
        "class_distribution": {
          "2": 0.4205607476635514,
          "1": 0.5794392523364486
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_width >= 1": {
        "leaf": true,
        "class_distribution": {
          "1": 0.594059405940594,
          "2": 0.40594059405940597
        }
      }
    }
  }
]
```

In [53]:
def test_99f31e()
  iris = load_iris_dataset()
  model = RandomForestLearner.new 2, num_trees: 3, max_depth: 2, min_size: 25
  model.train iris
  trees = model.trees

  puts "Random Forest 3 trees, 2 levels", model.to_s

  assert_equal 3, trees.size
end
test_99f31e()

Random Forest 3 trees, 2 levels
[
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_width >= 1": {
        "leaf": true,
        "class_distribution": {
          "1": 0.4444444444444444,
          "2": 0.5555555555555556
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      },
      "petal_width >= 1": {
        "leaf": true,
        "class_distribution": {
          "1": 0.4444444444444444,
          "2": 0.5555555555555556
        }
      }
    }
  },
  {
    "leaf": false,
    "split": "Numeric[petal_width <=> 1]",
    "children": {
      "petal_width < 1": {
        "leaf": true,
        "class_distribution": {
          "0": 1.0
        }
      

## Question 6.3 (5 Points)

Implement the ```predict``` method in the ```RandomForestLearner```. Define the score for a random forest as follows:

# $P(c \mid t_1, \dots ,t_k) = \frac{1}{k} \sum_{i} P(c \mid t_i)$

where $t_i$ is the $i$th tree and $P(c \mid t_i)$ is the score for the $i$th tree.

In [54]:
class RandomForestLearner
  attr_accessor :positive_class_label
  
  def evaluate eval_dataset
    examples = eval_dataset["data"]
    examples.map do |example|
      score = predict(example)

      label = example["label"] == @positive_class_label ? 1 : 0
      [score, label]
    end
  end
  
  def predict example
    # BEGIN YOUR CODE
    a = []
    @trees.each do |n|
       a.append(n.predict(example))
   end

    return mean(a)
    
    #END YOUR CODE
  end
end


:predict

In [ ]:
def test_99f31e()
  german_credit = load_german_credit_dataset()
  learner = RandomForestLearner.new 1, num_trees: 11, min_size: 100, max_depth: 10
  learner.train german_credit
  assert_equal 11, learner.trees.size

  scores = learner.evaluate german_credit
  metric = AUCMetric.new
  fp, tp, auc = metric.roc_curve scores
  
  assert_true auc > 0.79, "AUC should be better than 0.79"
  puts "AUC = #{auc}"
  
  plot = Daru::DataFrame.new({x: fp, y: tp}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "False Positive Rate"
    plot.y_label "True Positive Rate"
  end
  plot.add(:line, [0,1], [0,1]).color(:gray)
  plot.show()
end
test_99f31e()

Test::Unit::AssertionFailedError: AUC should be better than 0.79.
<true> expected but was
<false>

In the tests above, we calculated the AUC for a single decision tree and a random forest, using the same tree depth and minimum leaf size. Is the training AUC better on Random Forest and Decision Tree?

**A**. We cannot compare AUC across different models.

**B**. Both Single Decision Tree and Random Forest perform the same.

**C**. Single Decision Tree has better training AUC that a Random Forest.

**D**. Random Forest has better training AUC that a Single Decision Tree.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answer is "Z", write the following:_

```ruby
def answer_0b47d3()
    %w(Z)
end
```

In [56]:
def answer_0b47d3()
  # BEGIN YOUR CODE
  %w(D)
  #END YOUR CODE
end

:answer_0b47d3

In [57]:
assert_not_nil answer_0b47d3()

# Question 7.1 (4 Points)

Paste **your** implementation of ```cross_validate``` from previous assignments. 

In [58]:
def cross_validate dataset, folds, &block
  examples = dataset["data"]
  fold_size = examples.size / folds
  # BEGIN YOUR CODE

  
 

  folds.times do |fold|


      test_data = dataset.clone
      test_data["data"] = test_data["data"][fold*fold_size,fold_size]  ##CV training examples

      train_data = dataset.clone
      train_data["data"] = train_data["data"] - test_data["data"]  ##CV testing examples


    yield train_data, test_data, fold
  end
  
  
  
  
  
  
  
  #END YOUR CODE
end

:cross_validate

## Question 7.2 (5 points)

Using cross validation and the generic ```Learner```s and ```Metric```s you implemented, we can build a generic tool for parameter selection. The next few methods are implemented for you so that you can interpret the results. 


In [59]:
def cross_validation_model_performance dataset, folds, learners, metric    
  learners.map do |learner|
    tr_metrics = []
    te_metrics = []
    puts "#{folds}-fold CV: #{learner.class.name}, parameters: #{learner.parameters}"
    cross_validate dataset, folds do |train_dataset, test_dataset|
      learner.train train_dataset
      train_scores = learner.evaluate train_dataset
      test_scores = learner.evaluate test_dataset      
      tr_metrics << metric.apply(train_scores)
      te_metrics << metric.apply(test_scores)
    end
    {
      "learner" => learner.class.name, "parameters" => learner.parameters, "folds" => folds,
      "mean_train_metric" => mean(tr_metrics), "stdev_train_metric" => stdev(tr_metrics),
      "mean_test_metric" => mean(te_metrics), "stdev_test_metric" => stdev(te_metrics),
    }
  end
end


:cross_validation_model_performance

In [60]:
def best_performance_by_learner stats  
  stats.group_by {|s| s["learner"]}.map do |g_s|
    learner, learner_stats = g_s
    best_parameters = learner_stats.max_by {|l| l["mean_test_metric"]}    
    [learner, best_parameters]
  end.to_h
end

def parameter_search learners, dataset
  metric = AUCMetric.new  
  stats = cross_validation_model_performance dataset, 5, learners, metric
  best_by_learner = best_performance_by_learner stats  
  puts JSON.pretty_generate(best_by_learner)

  assert_equal learners.size, stats.size
  assert_true(stats.all? {|s| a = s["mean_train_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  assert_true(stats.all? {|s| a = s["mean_test_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  
  df = Daru::DataFrame.new(stats) 
end

:parameter_search

We will select the best-performing model on the iris using a single decision tree with various ```min_size``` parameter values. 

In [61]:
def test_6774f4()
  iris = load_iris_dataset()
  limit = iris["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    DecisionTreeLearner.new(1, min_size: s, max_depth: max_depth)
  end

  parameter_search learners, iris
end
test_6774f4()

5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>1, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>2, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>5, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>10, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>15, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>20, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>25, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>50, "max_depth"=>50}
5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>100, "max_depth"=>50}
{
  "DecisionTreeLearner": {
    "learner": "DecisionTreeLearner",
    "parameters": {
      "min_size": 5,
      "max_depth": 50
    },
    "folds": 5,
    "mean_train_metric": 0.9989848305487878,
    "stdev_train_metric": 4.787656956139239e-07,
    "mean_test_metric": 0.9492203554958973,
    "stde

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,DecisionTreeLearner,"{""min_size""=>1, ""max_depth""=>50}",5,1.0,0.0,0.9151294464049883,0.00731899526019191
1,DecisionTreeLearner,"{""min_size""=>2, ""max_depth""=>50}",5,1.0,0.0,0.9151294464049883,0.00731899526019191
2,DecisionTreeLearner,"{""min_size""=>5, ""max_depth""=>50}",5,0.9989848305487878,4.787656956139239e-07,0.9492203554958973,0.002930380251502318
3,DecisionTreeLearner,"{""min_size""=>10, ""max_depth""=>50}",5,0.99780235631668,1.946416625259581e-06,0.9474104007447661,0.003146954411830592
4,DecisionTreeLearner,"{""min_size""=>15, ""max_depth""=>50}",5,0.99780235631668,1.946416625259581e-06,0.9474104007447661,0.003146954411830592
5,DecisionTreeLearner,"{""min_size""=>20, ""max_depth""=>50}",5,0.99780235631668,1.946416625259581e-06,0.9474104007447661,0.003146954411830592
6,DecisionTreeLearner,"{""min_size""=>25, ""max_depth""=>50}",5,0.99780235631668,1.946416625259581e-06,0.9474104007447661,0.003146954411830592
7,DecisionTreeLearner,"{""min_size""=>50, ""max_depth""=>50}",5,0.9785729866192892,7.493969689611498e-05,0.9210837861828575,0.0049687602299819935
8,DecisionTreeLearner,"{""min_size""=>100, ""max_depth""=>50}",5,0.7621352062907693,0.00017650641242245434,0.7390360800190522,0.0016407766671213452


Select the best-performing model using RandomForest on the iris dataset with various ```min_size``` parameters.

In [62]:
def test_f9aacd()
  iris = load_iris_dataset()
  limit = iris["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  num_trees = 11
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    RandomForestLearner.new(1, num_trees: num_trees, min_size: s, max_depth: max_depth)
  end

  parameter_search learners, iris
end
test_f9aacd()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>1, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>2, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>5, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>10, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>15, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>20, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>25, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>50, "max_depth"=>50}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>100, "max_depth"=>50}
{
  "RandomForestLearner": {
    "learner": "RandomForestLearner",
    "parameters": {
      "num_trees": 11,
      "min_size": 1,
      "max_depth

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>1, ""max_depth""=>50}",5,0.979792019765371,0.0002394771057560725,0.9414766286345234,0.0028698977105261108
1,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>2, ""max_depth""=>50}",5,0.979792019765371,0.0002394771057560725,0.9414766286345234,0.0028698977105261108
2,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>5, ""max_depth""=>50}",5,0.9793462550008188,0.0002342987758520636,0.9414766286345234,0.0028698977105261108
3,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>10, ""max_depth""=>50}",5,0.9825804480784497,0.00029636300204063064,0.9414766286345234,0.0028698977105261108
4,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>15, ""max_depth""=>50}",5,0.9816315690649471,0.00028854926897181886,0.9414766286345234,0.0028698977105261108
5,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>20, ""max_depth""=>50}",5,0.9795345543763269,0.0002813183424622847,0.9380675377254326,0.0024292284814298477
6,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>25, ""max_depth""=>50}",5,0.9795345543763269,0.0002813183424622847,0.9380675377254326,0.0024292284814298477
7,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>50, ""max_depth""=>50}",5,0.9643153706195013,0.0002369649663985444,0.9396296520816645,0.0019001911551786382
8,RandomForestLearner,"{""num_trees""=>11, ""min_size""=>100, ""max_depth""=>50}",5,0.7621352062907693,0.00017650641242245434,0.746275899023577,0.0008113936746182847


We will select the best-performing model on the german credit dataset using a single decision tree with various ```min_size``` parameter values. 

In [ ]:
def test_0678f4()
  german_credit = load_german_credit_dataset()
  limit = german_credit["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
    DecisionTreeLearner.new(1, min_size: s, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_0678f4()

5-fold CV: DecisionTreeLearner, parameters: {"min_size"=>1, "max_depth"=>50}


NoMethodError: undefined method `children' for nil:NilClass

In [ ]:
def test_f3ac87()
  german_credit = load_german_credit_dataset()
  limit = german_credit["data"].size
  min_sizes = [1, 2, 5, 10, 15, 20, 25, 50, 100, 200, 250, 500]
  max_depth = 50  
  num_trees = 11
  min_sizes.select! {|s| s < limit}

  learners = min_sizes.map do |s|
      RandomForestLearner.new(1, num_trees: num_trees, min_size: s, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_f3ac87()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>1, "max_depth"=>50}


NoMethodError: undefined method `children' for nil:NilClass

Next, let's see the effect of increasing the number of trees for the iris dataset

In [ ]:
def test_10cb92()
  german_credit = load_iris_dataset()
  min_size = 50
  max_depth = 3
  num_trees = [1,2,3,5,7,9,11, 21, 51, 101, 151, 201]
  
  learners = num_trees.map do |s|
    RandomForestLearner.new(1, num_trees: s, min_size: min_size, max_depth: max_depth)
  end
  parameter_search learners, german_credit
end
test_10cb92()

5-fold CV: RandomForestLearner, parameters: {"num_trees"=>1, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>2, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>3, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>5, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>7, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>9, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>11, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>21, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>51, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>101, "min_size"=>50, "max_depth"=>3}
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>151, "m

Interrupt: 

## Question 7.3 (2 Points)

In ```test_6774f4``` above, we test different min sizes for the iris dataset. Considering the results for that test, which of the following are true?

**A**. The best performance on the testing set was ```min_size = 150```.

**B**. The best performance on the testing set was ```min_size = 1```.

**C**. The best performance on the training set was ```min_size = 1```.

**D**. The best performance on the testing set was neither 150 nor 1.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers are "E" and "F", write the following:_

```ruby
def answer_fa007f()
    %w(E F)
end
```

In [66]:
def answer_fa007f()
  # BEGIN YOUR CODE
  %w(C D)
  #END YOUR CODE
end

:answer_fa007f

In [67]:
assert_not_nil answer_fa007f()

## Question 7.4 (2 Points)

In ```test_0678f4``` and ```test_f3ac87``` above, we test different min sizes for the german credit dataset for a single Decision Tree and a random forest. Considering the results for these tests, which of the following are true?

**A**. RandomForest has better average test AUC than a single Decision Tree.

**B**. RandomForest has better average train AUC than a single Decision Tree.

**C**. RandomForest has lower standard deviation test AUC than a single Decision Tree.

**D**. RandomForest has higher standard deviation test AUC than a single Decision Tree.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers are "E" and "F", write the following:_

```ruby
def answer_545512()
    %w(E F)
end
```

In [68]:
def answer_545512()
  # BEGIN YOUR CODE
  %w(A C)
  #END YOUR CODE
end

:answer_545512

In [69]:
assert_not_nil answer_545512()

## Question 7.5 (2 Points)

In ```test_10cb92``` above, we tested different number of trees for a random forest on the iris dataset. Considering the results for these tests, which of the following are true?

**A**. Increasing the number of trees makes no difference for the iris dataset in this test.

**B**. RandomForest testing AUC usually does not become worse with increasing number of trees.

**C**. The average testing AUC is monotonically decreasing as the number of trees increases.

**D**. Increasing the number of trees led to overfitting for the iris dataset.


_(Instructions) In the function below, return an array of the upper-case letters indicating your answer (zero or more). For example, if you think that the answers is "Z", write the following:_

```ruby
def answer_ce174b()
    %w(Z)
end
```

In [70]:
def answer_ce174b()
  # BEGIN YOUR CODE
  %w(B)
  #END YOUR CODE
end

:answer_ce174b

In [71]:
assert_not_nil answer_ce174b()